In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

# open txt file
file_object = open('/content/drive/My Drive/data/covid-label.txt')
try:
    file_content = file_object.read()
finally:
    file_object.close()
# split by ",""
result = file_content.split(',')

label = []
for i in result:
   label.append(i)

label = list(map(int, label))

Mounted at /content/drive


In [ ]:
import json
import string
from keras.preprocessing.text import Tokenizer
import pandas as pd
import numpy as np
import re
import nltk
import pip
from nltk import collections
from nltk.tag import pos_tag
from nltk.corpus import wordnet
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm, metrics
from sklearn.metrics import precision_recall_fscore_support
from nltk.stem import LancasterStemmer, WordNetLemmatizer

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')

# clean data
default_stopwords = set(nltk.corpus.stopwords.words('english'))
# will have to add the following custom
custom_stopwords = {"http://", "rt", "co", "https://", "www", "@"}
all_stopwords = default_stopwords | custom_stopwords
eng_stemmer = nltk.stem.SnowballStemmer('english')
tt = TweetTokenizer()
lemmatizer = WordNetLemmatizer()

def clean_data(text):
    # print("Started preprocessing!")
    # text = df['reply_text'].apply(str)
    text = re.sub(r'((www\.[^\s]+)|(https?://[^\s]+))', '', text)
    text = re.sub(r'@[^\s]+', '', text)
    text = tt.tokenize(text)
    # reove single character words
    text = [word for word in text if len(word) > 1]
    # convert to lower case
    text = [word.lower() for word in text]
    # removing numbers
    text = [word for word in text if not word.isnumeric()]
    # stem the words
    text = [lemmatizer.lemmatize(word) for word in text]
    # remove stopwords
    text = [word for word in text if word not in default_stopwords]
    text = [word for word in text if nltk.re.match(r".*[a-z]+.*", word)]
    text = " ".join(text)
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
def process_json_data(read_data_filepath, label):
  with open(read_data_filepath, 'r') as f:
    source_id = []
    source_text = []
    reply_text = []
    for line in f:
      line = json.loads(line)
      source_id.append(clean_data(line[0]['id_str']))
      source_text.append(clean_data(line[0]['text']))
      tmp = []
      for i in range(1, len(line)):
        tmp.append(clean_data(line[i]['text']))
      reply_text.append(tmp)
    df_data = pd.DataFrame({'id':source_id,'source_text':source_text, 'reply_text':reply_text, 'label':label})
    return df_data[['id', 'source_text', 'reply_text', 'label']]

df_covid = process_json_data('/content/drive/My Drive/data/covid.data.jsonl', label)

# a list of events, and each event is a list of tweets (source tweet + reactions)
print("Number of test data =", len(df_covid["source_text"]))

Number of test data = 17458


In [ ]:
!pip install PaddlePaddle
import paddle
paddle.utils.run_check()

     |████████████████████████████████| 168.5MB 87kB/s 


2021-05-04 02:47:28,063 - WARNING - You are using GPU version PaddlePaddle, but there is no GPU detected on your machine. Maybe CUDA devices is not set properly.
 Original Error is Not compiled with CUDA


Running verify PaddlePaddle program ... 
PaddlePaddle works well on 1 CPU.
PaddlePaddle works well on 2 CPUs.
PaddlePaddle is installed successfully! Let's start deep learning with PaddlePaddle now.


In [ ]:
paddle.utils.run_check()

2021-05-04 02:47:28,150 - WARNING - You are using GPU version PaddlePaddle, but there is no GPU detected on your machine. Maybe CUDA devices is not set properly.
 Original Error is Not compiled with CUDA


Running verify PaddlePaddle program ... 
PaddlePaddle works well on 1 CPU.
PaddlePaddle works well on 2 CPUs.
PaddlePaddle is installed successfully! Let's start deep learning with PaddlePaddle now.


In [ ]:
!pip install Senta

     |████████████████████████████████| 184kB 5.2MB/s 
     |████████████████████████████████| 12.2MB 18.1MB/s 
     |████████████████████████████████| 1.5MB 42.3MB/s 
     |████████████████████████████████| 1.0MB 37.4MB/s 
     |████████████████████████████████| 5.4MB 41.0MB/s 
  Created wheel for nltk: filename=nltk-3.4.5-cp37-none-any.whl size=1449906 sha256=54783a9f883ab17c90320edf266abd0c748ed94feee588c1a9aecb42145a47ee
  Stored in directory: /root/.cache/pip/wheels/96/86/f6/68ab24c23f207c0077381a5e3904b2815136b879538a24b483
Successfully built nltk
ERROR: xarray 0.15.1 has requirement numpy>=1.15, but you'll have numpy 1.14.5 which is incompatible.
ERROR: tifffile 2021.4.8 has requirement numpy>=1.15.1, but you'll have numpy 1.14.5 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement numpy~=1.19.2, but you'll have numpy 1.14.5 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement six~=1.15.0, but you'll have six 1.11.0 which is incompatible.
ERROR: spacy 2.2.4 h

In [ ]:
from senta import Senta

In [ ]:
my_senta = Senta()

# 获取目前支持的情感预训练模型, 我们开放了以ERNIE 1.0 large(中文)、ERNIE 2.0 large(英文)和RoBERTa large(英文)作为初始化的SKEP模型
print(my_senta.get_support_model()) # ["ernie_1.0_skep_large_ch", "ernie_2.0_skep_large_en", "roberta_skep_large_en"]

# 获取目前支持的预测任务
print(my_senta.get_support_task()) # ["sentiment_classify", "aspect_sentiment_classify", "extraction"]

# 选择是否使用gpu
use_cuda = False # 设置True or False

['ernie_1.0_skep_large_ch', 'ernie_2.0_skep_large_en', 'roberta_skep_large_en']
['sentiment_classify', 'aspect_sentiment_classify', 'extraction']


In [ ]:
my_senta.init_model(model_class="roberta_skep_large_en", task="sentiment_classify", use_cuda=False)

In [ ]:
texts = ["a perfect film"]
result = my_senta.predict(texts)
print(result)

[('a perfect film', 'positive')]


In [ ]:
source_data = df_covid['source_text'].values.tolist()
# type(source_data)

In [ ]:
source_senti = []
tmpList = []
j = 17400
while j < 17459:
  source_senti = []
  tmpList = source_data[j:(j+100)]
  tmp = my_senta.predict(tmpList)
  for i in tmp:
    source_senti.append(1 if i[1] == 'positive' else 0)
  df = pd.DataFrame(source_senti)
  df.to_csv("/content/drive/My Drive/data/source_senti.csv", mode='a', index=False, header=False)
  # df.to_csv("source_senti.csv", mode='a', index=False, header=False)
  j += 100
  print(j)


17500


In [ ]:
# df = pd.DataFrame(source_senti)
# df.to_csv('source_senti.csv', index=False, heder=False)